# REINFORCE

---

In this notebook, we will train REINFORCE with OpenAI Gym's Cartpole environment.

### 1. Import the Necessary Packages

In [4]:
import gym
gym.logger.set_level(40) # suppress warnings (please remove if gives error)
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

### 2. Define the Architecture of the Policy

In [5]:
env = gym.make('Acrobot-v1')
env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size).to(device)
        self.fc2 = nn.Linear(h_size, h_size).to(device)
        self.fc3 = nn.Linear(h_size, a_size).to(device)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=-1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

observation space: Box(6,)
action space: Discrete(3)


In [ ]:
state = env.reset()
state = torch.from_numpy(state).float().to(device)
policy = Policy()
print("state", state)
prob = policy.forward(state)
print("prob",prob)
g = Categorical(prob)
print("categorical ",g)
print(g.sample())

In [ ]:
a = [x for x in range(1,10)]
b = [x for x in range(20,30)]
a = [torch.from_numpy(np.array(x)) for x in a]
a = torch.from_numpy(np.array(a))

### 3. Train the Agent with REINFORCE

In [6]:
policy = Policy(6,16,3).to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def reinforce(n_episodes=4000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        discounts = [gamma**i for i in range(len(rewards)+1)]
        R = sum([a*b for a,b in zip(discounts, rewards)])
        
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()
        
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores
    
scores = reinforce()

Episode 100	Average Score: -212.32
Episode 200	Average Score: -111.25
Episode 300	Average Score: -91.76
Episode 400	Average Score: -101.22
Episode 500	Average Score: -105.87
Episode 600	Average Score: -99.45
Episode 700	Average Score: -107.52
Episode 800	Average Score: -111.99
Episode 900	Average Score: -112.48
Episode 1000	Average Score: -87.78
Episode 1100	Average Score: -129.95
Episode 1200	Average Score: -103.75
Episode 1300	Average Score: -87.76
Episode 1400	Average Score: -96.87
Episode 1500	Average Score: -92.15
Episode 1600	Average Score: -82.78
Episode 1700	Average Score: -95.07
Episode 1800	Average Score: -91.60
Episode 1900	Average Score: -94.84
Episode 2000	Average Score: -106.00
Episode 2100	Average Score: -153.89
Episode 2200	Average Score: -116.15
Episode 2300	Average Score: -90.58
Episode 2400	Average Score: -96.53
Episode 2500	Average Score: -88.98
Episode 2600	Average Score: -89.25
Episode 2700	Average Score: -90.14
Episode 2800	Average Score: -94.77
Episode 2900	Aver

### 4. Plot the Scores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Watch a Smart Agent!

In [ ]:
env = gym.make('CartPole-v0')

state = env.reset()
for t in range(1000):
    action, logprob = policy.act(state)
    print('\r action {} with log prob {}'.format(action, logprob), end = ' ')
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 

env.close()